# Example - Comparison of sequences with CSV database

In [ ]:
import numpy as np
import pandas as pd

from dtw import DTW
from dtw.tasks.search_free_ends import brute_force_free_ends_search
from dtw.metrics import mixed_dist
from dtw.visu import plot_sequences
from dtw.visu import plot_aligned_sequences

Load the test dataset in `DB_eval_v1`:

In [ ]:
from dtw.util import default_test_db
gt_csv, pred_csv = default_test_db()

In [ ]:
gt_df = pd.read_csv(gt_csv)
pred_df = pd.read_csv(pred_csv)

In [ ]:
gt_df

In [ ]:
print(gt_df["PlantID"].unique())  # list the available "PlantID"

In [ ]:
plant_id = "Col0_12_10_2018_A"  # select one for alignment and comparison

In [ ]:
seq_ref = np.array([gt_df[gt_df["PlantID"] == plant_id]["angles"],
                   gt_df[gt_df["PlantID"] == plant_id]["Internodes"]]).T
seq_test = np.array([pred_df[pred_df["PlantID"] == plant_id]["angles"],
                   pred_df[pred_df["PlantID"] == plant_id]["Internodes"]]).T

## Initialize the DTW object

To performs this alignment, we use the "merge-split" constraint to account for organ under or over segmentation.
We also use a "mixed distance" that we need to initialize.

### Mixed-distance initialization

In [ ]:
max_ref = np.max(seq_ref[:, 1])
max_test = np.max(seq_test[:, 1])
# Variables associated to 'mixed-distance':
mtype = [True, False]
mspread = [1, max(max_ref, max_test)]
mweight = [0.5, 0.5]

### Dynamic Time Warpping algorithm

In [ ]:
dtw = DTW(seq_test, seq_ref, names=['angles'], constraints='merge_split', ldist=mixed_dist,
          mixed_type=mtype, mixed_spread=mspread, mixed_weight=mweight)

In [ ]:
plot_sequences(dtw)

### Free-ends brute force search

In [ ]:
free_ends, norm_dist = brute_force_free_ends_search(dtw)
print(f"Found free-ends {free_ends} with a normalized distance of {norm_dist}.")

### Alignment with selected free-ends

Set the free-ends values to use in `run` method:

In [ ]:
dtw.free_ends = free_ends

In [ ]:
ndist, path, length, ndistarray, backpointers = dtw.run()

In [ ]:
dtw.summarize()

Now we can represent the aligned sequences using the `plot_aligned_sequences` method:

In [ ]:
plot_aligned_sequences(dtw)